In [1]:
from urllib2 import urlopen
from zipfile import ZipFile
from StringIO import StringIO
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import fiona
import fiona.crs
import pandas as pd
import requests
from shapely.geometry import Point
from numpy.random import RandomState, uniform
import numpy as np
import pandas as pd

## Getting Census Variables

In [7]:
def get_census_variables(df,variables,variable_labels=None):
 
    #identify geography fields - concatenate them into a fips code to be set as index and then delete them
    geo_fields = [x for x in df.columns if x not in ['NAME'] + variables]
    df.index = df[geo_fields].apply(lambda row: ''.join(map(str, row)), 1)
    df.index.name = 'FIPS'
    df = df.drop(geo_fields, 1)
    
    if variable_labels:
        df = df.rename(columns = dict(zip(variables, variable_labels)))
    
    #convert data numeric 
    df = df.applymap(lambda x:pd.to_numeric(x, errors='ignore'))
    return df

In [8]:
df = pd.read_csv('Data/Demographic Data/ny_acs_2018')
variables = ['asian_pop','black_pop','hispanic_pop','other_race_pop','white_pop','amerindian_pop','two_or_more_races_pop']
variable_names = ['Asian','Black','Hispanic','Other_','White','AI/AN','Two Plus']
get_census_variables(df,variables,variable_names)

,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,armed_forces,civilian_labor_force
0,350399407003,2014-01-01,1222,447,599,623,45.9,72,12,31,...,223,16,64,115,NaN,NaN,NaN,NaN,NaN,NaN
1,350390003001,2014-01-01,1256,672,473,783,41.3,0,0,107,...,170,92,0,290,NaN,NaN,NaN,NaN,NaN,NaN
2,350399408001,2014-01-01,1626,499,717,909,50.1,17,28,49,...,283,127,37,184,NaN,NaN,NaN,NaN,NaN,NaN
3,350390002002,2014-01-01,1995,588,916,1079,31.3,0,120,62,...,186,251,54,174,NaN,NaN,NaN,NaN,NaN,NaN
4,350399408002,2014-01-01,2398,821,1158,1240,48.3,50,60,44,...,414,121,206,348,NaN,NaN,NaN,NaN,NaN,NaN
5,350399407002,2014-01-01,1287,431,609,678,38.9,52,62,14,...,252,24,50,131,NaN,NaN,NaN,NaN,NaN,NaN
6,350390005001,2014-01-01,1380,427,708,672,49.5,108,92,47,...,353,101,49,155,NaN,NaN,NaN,NaN,NaN,NaN
7,350399441004,2014-01-01,1424,512,747,677,43.3,58,44,76,...,269,61,17,203,NaN,NaN,NaN,NaN,NaN,NaN
8,350399410002,2014-01-01,1349,201,612,737,27.5,80,111,65,...,220,28,22,158,NaN,NaN,NaN,NaN,NaN,NaN
9,350399441003,2014-01-01,1445,409,780,665,40.8,55,22,50,...,403,43,22,216,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
def get_census_variables(year, dataset, geography, area, variables, variable_labels = None):
    """Wraps the Census API and returns a DataFrame of Census Data
    Parameters
    ----------
    year : integer
        Year representing the dataset vintage 
    dataset : string
        the name of the dataset (https://api.census.gov/data.html)
    geography : string
        the census geography
    area : dictionary
        dictionary contains the FIPS codes at each nested geographic level. For example "{'county':'001', 'state':'06'}"
    variables : list
        list of the variables to be extracted
    variable_labels : list
        optional to relabel the variable names. Must be same length as "variables"
    """
    
    base_url = 'https://api.census.gov/data/{}/{}'.format(year, dataset)
    
    #define parameters
    get_parameter = ','.join(['NAME'] + variables)
    for_parameter = '{}:*'.format(geography)
    in_paramater = '+'.join([k+':'+v for (k,v) in area.items()])

    parameters = {'get' : get_parameter, 
                  'for' : for_parameter,
                  'in' : in_paramater}
    
    #make request specifiying url and parameters
    r = requests.get(base_url, params=parameters)
    print(r)
    
    #read json into pandas dataframe, specifying first row as column names
    data = r.json()
    df=pd.DataFrame(columns = data[0], data = data[1:])
    
    #identify geography fields - concatenate them into a fips code to be set as index and then delete them
    geo_fields = [x for x in df.columns if x not in ['NAME'] + variables]
    df.index = df[geo_fields].apply(lambda row: ''.join(map(str, row)), 1)
    df.index.name = 'FIPS'
    df = df.drop(geo_fields, 1)
    
    if variable_labels:
        df = df.rename(columns = dict(zip(variables, variable_labels)))
    
    #convert data numeric 
    df = df.applymap(lambda x:pd.to_numeric(x, errors='ignore'))
    return df

In [ ]:
'https://api.census.gov/data/2015/acs5/'

## Spatial Mapping Functions

In [16]:
def gen_random_points_poly(poly, num_points, seed = None):
    """
    Returns a list of N randomly generated points within a polygon. 
    """
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    i=0
    while len(points) < num_points:
        s=RandomState(seed+i) if seed else RandomState(seed)
        random_point = Point([s.uniform(min_x, max_x), s.uniform(min_y, max_y)])
        if random_point.within(poly):
            points.append(random_point)
        i+=1
    return points

In [35]:
def gen_points_in_gdf_polys(geometry, values, points_per_value = None, seed = None):
    """
    Take a GeoSeries of Polygons along with a Series of values and returns randomly generated points within
    these polygons. Optionally takes a "points_per_value" integer which indicates the number of points that 
    should be generated for each 1 value.
    """
    if points_per_value:
        new_values = (values/points_per_value).astype(int)
    else:
        new_values = values
    new_values = new_values[new_values>0]
    g = gpd.GeoDataFrame(data = {'vals':new_values}, geometry = geometry)
    print(g.shape)
    a = g.apply(lambda row: tuple(gen_random_points_poly(row['geometry'], row['vals'], seed)),1)
    b = gpd.GeoSeries(a.apply(pd.Series).stack(), crs = geometry.crs)
    b.name='geometry'
    return b

In [36]:
def zip_shp_to_gdf(zip_file_name):
    """
    Returns a GeoDataFrame from a URL for a zipped Shapefile
    """
    zipfile = ZipFile(StringIO(urlopen(zip_file_name).read()))
    filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx']\
                 if y.endswith(ending)] 
    dbf, prj, shp, shx = [StringIO(zipfile.read(filename)) for filename in filenames]
    r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
    
    attributes, geometry = [], []
    field_names = [field[0] for field in r.fields[1:]]  
    for row in r.shapeRecords():  
        geometry.append(shape(row.shape.__geo_interface__))  
        attributes.append(dict(zip(field_names, row.record)))  
        
    proj4_string = fiona.crs.from_epsg(2136)
    gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)
    return gdf

In [39]:
def gen_count_dot_density_map(county, pts_per_person = 300, 
                              epsg = 2163, seed=10,
                              dot_transparency=0.4, figsize=(12,12), 
                              ax=None, legend=True):
    """
    Wraps previous functions and generates population dot density maps for a specified county by race
    
    """
    #read in fips to county name relationship file
    fips = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',
                   header=None, dtype={1:np.object, 2:np.object})
    fips['name']=fips[3]+', '+fips[0]
    fips['fips']=fips[1]+fips[2]
    
    #get name from fips if fips specified
    if county.isdigit():
        lookup = fips.set_index('fips')['name']
        county_fips = county
        name = lookup[county_fips]
    #get fips from name if name specified
    else:
        lookup = fips.set_index('name')['fips']
        name = county
        county_fips = lookup[name]
        print(county_fips)
    
    
    #get geodataframe of block group shapefile
    bgfile_name = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_{}_bg_500k.zip'.format(county_fips[:2])
    bg_geo = zip_shp_to_gdf(bgfile_name)
    
    #subset to those that are in the county and project it to the CRS
    bg_geo=bg_geo[bg_geo['GEOID'].str[:5]==county_fips].to_crs(epsg=epsg).set_index("GEOID")['geometry']
    print(bg_geo)
    
    #specify variable list and variable names for the census api function
    varlist = ['B03002_003E', 
               'B03002_012E',
               'B03002_004E', 
               'B03002_006E',
               'B03002_005E',
               'B03002_007E',
               'B03002_008E',
               'B03002_009E']
    names = ['White',
             'Hispanic',
             'Black',
             'Asian',
             'AI/AN',
             'NH/PI',
             'Other_',
             'Two Plus']
    
    area = {'county':county_fips[2:],'state':county_fips[:2]}
    print(area)
    #read in block group level census variables
    dems = get_census_variables(2015, 'acs5', 'block group', area, varlist, names)
    #Calculate other as sum of those not in the 4 most populated race categories
    dems['Other']=dems[['AI/AN', 'NH/PI','Other_', 'Two Plus']].sum(1)
    
    #Calculate county boundaries as the union of block groups 
    union = gpd.GeoSeries(bg_geo.unary_union)
    
    #if axes object is specified, plot to this axis, otherwise create a new one
    if ax:
        union.plot(color='white', figsize=figsize, ax=ax)
    else:
        ax = union.plot(color='white', figsize=figsize)
   
    #set aspect equal and add title if specified
    ax.set(aspect='equal', xticks=[], yticks=[])
    #set title as county name
    ax.set_title(name, size=15)
    
    #annotate the dot per person ratio
    ax.annotate("1 dot = {} people".format(pts_per_person), 
                xy=(.5, .97), xycoords='axes fraction', horizontalalignment='center',
                fontsize = 12)
        
    #loop each race category and generate points for each within each block group 
    list_of_point_categories=[]
    for field in ['White','Hispanic','Black','Asian','Other']:           
        ps=gpd.GeoDataFrame(gen_points_in_gdf_polys(geometry = bg_geo, values=dems[field],
                             points_per_value = pts_per_person, seed=None))
        ps['field']=field
        list_of_point_categories.append(ps)
    all_points=gpd.GeoDataFrame(pd.concat(list_of_point_categories))
    all_points.plot(ax=ax, markersize=2, alpha=dot_transparency, 
              column='field', categorical=True, legend=legend)

    return ax

In [40]:
gen_count_dot_density_map('Kings County, NY', pts_per_person=200)

36047
GEOID
360470276003    POLYGON ((9024673.813 1254016.173, 9024673.813...
360470547003    POLYGON ((9024673.798 1254016.212, 9024673.798...
360470505002    POLYGON ((9024673.800 1254016.219, 9024673.801...
360470218001    POLYGON ((9024673.809 1254016.182, 9024673.810...
360470820002    POLYGON ((9024673.810 1254016.202, 9024673.810...
360470293002    POLYGON ((9024673.805 1254016.217, 9024673.806...
360470439002    POLYGON ((9024673.806 1254016.227, 9024673.806...
360470954001    POLYGON ((9024673.816 1254016.210, 9024673.816...
360470456001    POLYGON ((9024673.813 1254016.190, 9024673.814...
360470544001    POLYGON ((9024673.817 1254016.190, 9024673.817...
360470228001    POLYGON ((9024673.809 1254016.188, 9024673.810...
360470525002    POLYGON ((9024673.799 1254016.214, 9024673.798...
360470966001    POLYGON ((9024673.818 1254016.212, 9024673.818...
360470477002    POLYGON ((9024673.799 1254016.223, 9024673.799...
360470311002    POLYGON ((9024673.807 1254016.212, 9024673.807..

TypeError: get_census_variables() takes at most 3 arguments (6 given)

In [72]:
from census_mapper import *

ImportError: No module named census_mapper

In [43]:
def zip_shp_to_gdf(zip_file_name):
    """
    Returns a GeoDataFrame from a URL for a zipped Shapefile
    """
    zipfile = ZipFile(StringIO(urlopen(zip_file_name).read()))
    filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx']\
                 if y.endswith(ending)] 
    dbf, prj, shp, shx = [StringIO(zipfile.read(filename)) for filename in filenames]
    r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
    
    attributes, geometry = [], []
    field_names = [field[0] for field in r.fields[1:]]  
    for row in r.shapeRecords():  
        geometry.append(shape(row.shape.__geo_interface__))  
        attributes.append(dict(zip(field_names, row.record)))  
        
    proj4_string = fiona.crs.from_epsg(2136)
    gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)
    return gdf

In [44]:
def gen_count_dot_density_map(county, pts_per_person = 300, 
                              epsg = 2163, seed=10,
                              dot_transparency=0.4, figsize=(12,12), 
                              ax=None, legend=True):
    """
    Wraps previous functions and generates population dot density maps for a specified county by race
    
    """
    #read in fips to county name relationship file
    fips = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',
                   header=None, dtype={1:np.object, 2:np.object})
    fips['name']=fips[3]+', '+fips[0]
    fips['fips']=fips[1]+fips[2]
    
    #get name from fips if fips specified
    if county.isdigit():
        lookup = fips.set_index('fips')['name']
        county_fips = county
        name = lookup[county_fips]
    #get fips from name if name specified
    else:
        lookup = fips.set_index('name')['fips']
        name = county
        county_fips = lookup[name]
        print(county_fips)
    
    
    #get geodataframe of block group shapefile
    bgfile_name = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_{}_bg_500k.zip'.format(county_fips[:2])
    bg_geo = zip_shp_to_gdf(bgfile_name)
    
    #subset to those that are in the county and project it to the CRS
    bg_geo=bg_geo[bg_geo['GEOID'].str[:5]==county_fips].to_crs(epsg=epsg).set_index("GEOID")['geometry']
    print(bg_geo)
    
    #specify variable list and variable names for the census api function
    varlist = ['B03002_003E', 
               'B03002_012E',
               'B03002_004E', 
               'B03002_006E',
               'B03002_005E',
               'B03002_007E',
               'B03002_008E',
               'B03002_009E']
    names = ['White',
             'Hispanic',
             'Black',
             'Asian',
             'AI/AN',
             'NH/PI',
             'Other_',
             'Two Plus']
    
    demo_data_2018 = pd.read_csv('Data/Demographic Data/ny_acs_2018')
    
    race_vars = ['asian_pop','black_pop','hispanic_pop','other_race_pop','white_pop','amerindian_pop','two_or_more_races_pop']
    race_names = ['Asian','Black','Hispanic','Other_','White','AI/AN','Two Plus']

    race_tab = demo_data_2018[['geo_id','asian_pop','black_pop','hispanic_pop','other_race_pop','white_pop','amerindian_pop','two_or_more_races_pop']]
    race_tab['Other']=race_tab[['other_race_pop', 'amerindian_pop','two_or_more_races_pop']].sum(1)
    
    #Calculate county boundaries as the union of block groups 
    union = gpd.GeoSeries(bg_geo.unary_union)
    
    #if axes object is specified, plot to this axis, otherwise create a new one
    if ax:
        union.plot(color='white', figsize=figsize, ax=ax)
    else:
        ax = union.plot(color='white', figsize=figsize)
   
    #set aspect equal and add title if specified
    ax.set(aspect='equal', xticks=[], yticks=[])
    #set title as county name
    ax.set_title(name, size=15)
    
    #annotate the dot per person ratio
    ax.annotate("1 dot = {} people".format(pts_per_person), 
                xy=(.5, .97), xycoords='axes fraction', horizontalalignment='center',
                fontsize = 12)
        
    #loop each race category and generate points for each within each block group 
    list_of_point_categories=[]
    for field in ['asian_pop','black_pop','hispanic_pop','other_race_pop','white_pop','amerindian_pop','two_or_more_races_pop']:           
        ps=gpd.GeoDataFrame(gen_points_in_gdf_polys(geometry = bg_geo, values=race_tab[field],
                             points_per_value = pts_per_person, seed=seed))
        ps['field']=field
        list_of_point_categories.append(ps)
    all_points=gpd.GeoDataFrame(pd.concat(list_of_point_categories))
    all_points.plot(ax=ax, markersize=2, alpha=dot_transparency, 
              column='field', categorical=True, legend=legend)

    return ax